In [ ]:
#bgr8 to jpeg format
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import time

import threading
import inspect
import ctypes
import HSV_Config

In [ ]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [ ]:
image_widget = widgets.Image(format='jpeg', width=300, height=300)
display(image_widget)

In [ ]:
image = cv2.VideoCapture(0)

image.set(3, 320)
image.set(4, 240)
image.set(5, 30)  #set frame
image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
image.set(cv2.CAP_PROP_BRIGHTNESS, 62) 
image.set(cv2.CAP_PROP_CONTRAST, 63)
image.set(cv2.CAP_PROP_EXPOSURE, 4800) 

ret, frame = image.read()
image_widget.value = bgr8_to_jpeg(frame)
update_hsv = HSV_Config.update_hsv()

In [ ]:
color_hsv  = {"red"   : ((0, 70, 72), (7, 255, 255)),
              "green" : ((54, 109, 78), (77, 255, 255)),
              "blue"  : ((92, 100, 62), (121, 251, 255)),
              "yellow": ((26, 100, 91), (32, 255, 255))}

In [ ]:
def TEST():
    global color_hsv
    while True:
        ret, frame = image.read()
        frame, binary = update_hsv.get_contours(frame,color_hsv)

        image_widget.value = bgr8_to_jpeg(frame)
        

In [ ]:
thread1 = threading.Thread(target=TEST)
thread1.setDaemon(True)
thread1.start()

In [ ]:
stop_thread(thread1)

In [ ]:
image.release()              #Release the object after use, otherwise when the next program needs to use the object, it will be unusable because it is occupied